# H & M --- Web Scraping

## LIBRARIES AND SETTINGS

In [1]:
import numpy as np
import pandas as pd
import requests
import logging
from datetime import datetime
from bs4 import BeautifulSoup
import sqlite3

In [2]:
# simulates a browser
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# home page
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

## 1. COLLECT DATA

### 1.1. Extraction the number of items on the page

In [3]:
# API request
page = requests.get(url, headers = headers)

# transform the html request into a beautiful soup object
soup = BeautifulSoup(page.text, 'html.parser')

# page size
itens = soup.find_all('h2')[2]

itens_shown = int(itens['data-items-shown'])
total_itens = int(itens['data-total'])

page_size = str(int(np.ceil(total_itens / itens_shown) * itens_shown))

# new url with the total amount of items
new_url = 'https://www2.hm.com/en_us/men/products/jeans.html?sort=stock&image-size=small&image=model&offset=0&page-size=' + page_size

### 1.2. Extraction of links for each product on the page

In [4]:
# API request
page = requests.get(new_url, headers = headers)

# transform the html request into a beautiful soup object
soup = BeautifulSoup(page.text, 'html.parser')

# identifies the products
product_item_li = soup.find_all('li', class_ = 'product-item')

# extract the details url
domain = 'https://www2.hm.com'
url = [domain + i.find('a')['href'] for i in product_item_li]

Exploring the H&M website, it was possible to see that each different color for a particular pair of pants had its own characteristics. For this reason, a routine to extract all the links of each color available for each product was made, in order to obtain the information

In [5]:
# creates the dataframe structure
color_info = pd.DataFrame(columns = ['color_url', 'color_id', 'color_name'])

for u in url:
    # API request
    page = requests.get(u, headers = headers)

    # transform the html request into a beautiful soup object
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # collects information about colors
    info = soup.find_all('a', class_ = 'filter-option')
    
    # auxiliary dataframe
    aux = pd.DataFrame(columns = ['color_url', 'color_id', 'color_name'])
    
    aux['color_url']  = [domain + i['href'] for i in info]
    aux['color_name'] = [i['title'] for i in info]
    aux['color_id']   = [i['data-articlecode'] for i in info]
    
    # contacts collected informations in a single dataframe
    color_info = pd.concat([color_info, aux], ignore_index = True)
    print(u)

# drop duplicates
color_info = color_info.drop_duplicates(keep = 'first').reset_index(drop = True)

https://www2.hm.com/en_us/productpage.1024256001.html
https://www2.hm.com/en_us/productpage.0985159001.html
https://www2.hm.com/en_us/productpage.1008549002.html
https://www2.hm.com/en_us/productpage.0875105024.html
https://www2.hm.com/en_us/productpage.0690449036.html
https://www2.hm.com/en_us/productpage.1024256002.html
https://www2.hm.com/en_us/productpage.1008549006.html
https://www2.hm.com/en_us/productpage.1004199004.html
https://www2.hm.com/en_us/productpage.0985159007.html
https://www2.hm.com/en_us/productpage.1004199001.html
https://www2.hm.com/en_us/productpage.1004199002.html
https://www2.hm.com/en_us/productpage.1008549001.html
https://www2.hm.com/en_us/productpage.0985159008.html
https://www2.hm.com/en_us/productpage.1024256003.html
https://www2.hm.com/en_us/productpage.1008110001.html
https://www2.hm.com/en_us/productpage.1024256004.html
https://www2.hm.com/en_us/productpage.0875105018.html
https://www2.hm.com/en_us/productpage.0690449056.html
https://www2.hm.com/en_us/pr

### 1.3. Extract information from each product 

In [ ]:
# API request
page = requests.get('https://www2.hm.com/en_us/productpage.0985159002.html', headers = headers)

# transform the html request into a beautiful soup object
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
# price and name
info1 = soup.find('div', class_ = 'inner')
if info1:
    print('ok')
    name = info1.find('h1').text
    price = info1.find('span', class_ = 'price-value').text
    print(f'name: {name}, price: {price}')
else:
    print('oh God')
    name = 'NaN'
    price = 'NaN'

In [ ]:
info2 = soup.find('div', class_ = 'details parbase')

if info2:
    aux2 = [list(filter(None, i.get_text().split('\n'))) for i in info2.find('dl').find_all('div')]
else:
    aux2 = [['Size', 'NaN'], ['Fit', 'NaN'], ['Composition', 'NaN', 'NaN'], 
            ['Art. No.', 'NaN'], ['Product safety', 'NaN']]
aux2

In [6]:
# empty dataframe
df_details = pd.DataFrame()

cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size']
df_pattern = pd.DataFrame(columns = cols)


for url in color_info.loc[:, 'color_url']:
    # API request
    page = requests.get(url, headers = headers)

    # transform the html request into a beautiful soup object
    soup = BeautifulSoup(page.text, 'html.parser')
     
    # price and name
    info1 = soup.find('div', class_ = 'inner')
    if info1:
        name = info1.find('h1').text
        price = info1.find('span', class_ = 'price-value').text
        print(f'url: {url}, name: {name}, price: {price}')
    else:
        name = 'NaN'
        price = 'NaN'
        print('oh God')
    
    # product features
    info2 = soup.find('div', class_ = 'details parbase')
    if info2:
        aux2 = [list(filter(None, i.get_text().split('\n'))) for i in info2.find('dl').find_all('div')]
    else:
        aux2 = [['Size', 'NaN'], ['Fit', 'NaN'], ['Composition', 'NaN', 'NaN'], 
                ['Art. No.', 'NaN'], ['Product safety', 'NaN']]
    
    # index
    line = color_info[color_info['color_url'] == url].index
    
    # put the data in the dataframe color_info
    color_info.loc[line, 'name']    = name
    color_info.loc[line, 'price']   = price
    color_info['web_scraping_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # reaname dataframe
    aux2 = pd.DataFrame(aux2).T
    aux2.columns = aux2.iloc[0]
    
    # delete first row
    aux2 = aux2.iloc[1:].fillna(method = 'ffill')
    
    # garantee the same number of columns
    aux2 = pd.concat([df_pattern, aux2], axis = 0)
    
    # all details products
    df_details = pd.concat([df_details, aux2], axis = 0)
       
# reset index    
df_details = df_details.reset_index(drop = True)

# merge
df = df_details.merge(color_info, left_on = 'Art. No.', right_on = 'color_id')

url: https://www2.hm.com/en_us/productpage.1024256001.html, name: Slim Jeans, price: 
                $19.99
url: https://www2.hm.com/en_us/productpage.1024256002.html, name: Slim Jeans, price: 
                $19.99
url: https://www2.hm.com/en_us/productpage.1024256003.html, name: Slim Jeans, price: 
                $19.99
url: https://www2.hm.com/en_us/productpage.1024256004.html, name: Slim Jeans, price: 
                $19.99
url: https://www2.hm.com/en_us/productpage.1024256005.html, name: Slim Jeans, price: 
                $19.99
url: https://www2.hm.com/en_us/productpage.1024256006.html, name: Slim Jeans, price: 
                $19.99
url: https://www2.hm.com/en_us/productpage.1024256007.html, name: Slim Jeans, price: 
                $19.99
url: https://www2.hm.com/en_us/productpage.1024256008.html, name: Slim Jeans, price: 
                $19.99
url: https://www2.hm.com/en_us/productpage.0985159001.html, name: Skinny Jeans, price: 
                $19.99
url: https://www2

url: https://www2.hm.com/en_us/productpage.0811993002.html, name: Regular Jeans, price: 
                $34.99
url: https://www2.hm.com/en_us/productpage.0811993003.html, name: Regular Jeans, price: 
                $18.99
url: https://www2.hm.com/en_us/productpage.0811993006.html, name: Regular Jeans, price: 
                $14.99
url: https://www2.hm.com/en_us/productpage.0811993007.html, name: Regular Jeans, price: 
                $22.99
url: https://www2.hm.com/en_us/productpage.0811993021.html, name: Regular Jeans, price: 
                $20.99
url: https://www2.hm.com/en_us/productpage.0811993022.html, name: Regular Jeans, price: 
                $16.99
url: https://www2.hm.com/en_us/productpage.0811993024.html, name: Regular Jeans, price: 
                $17.99
url: https://www2.hm.com/en_us/productpage.0811993025.html, name: Regular Jeans, price: 
                $15.99
url: https://www2.hm.com/en_us/productpage.0811993028.html, name: Regular Jeans, price: 
               

In [7]:
df

,Art. No.,Composition,Fit,Product safety,Size,color_url,color_id,color_name,name,price,web_scraping_date
0,1024256001,"Shell: Cotton 99%, Spandex 1%",Slim fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",https://www2.hm.com/en_us/productpage.10242560...,1024256001,Black,Slim Jeans,\r\n $19.99,2022-07-01 15:18:25
1,1024256001,"Pocket lining: Polyester 65%, Cotton 35%",Slim fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",https://www2.hm.com/en_us/productpage.10242560...,1024256001,Black,Slim Jeans,\r\n $19.99,2022-07-01 15:18:25
2,1024256002,"Shell: Cotton 99%, Spandex 1%",Slim fit,NaN,"The model is 189cm/6'2"" and wears a size 31/32",https://www2.hm.com/en_us/productpage.10242560...,1024256002,Light denim blue,Slim Jeans,\r\n $19.99,2022-07-01 15:18:25
3,1024256002,"Pocket lining: Polyester 65%, Cotton 35%",Slim fit,NaN,"The model is 189cm/6'2"" and wears a size 31/32",https://www2.hm.com/en_us/productpage.10242560...,1024256002,Light denim blue,Slim Jeans,\r\n $19.99,2022-07-01 15:18:25
4,1024256003,"Shell: Cotton 99%, Spandex 1%",Slim fit,NaN,"The model is 187cm/6'2"" and wears a size 31/32",https://www2.hm.com/en_us/productpage.10242560...,1024256003,Light denim blue,Slim Jeans,\r\n $19.99,2022-07-01 15:18:25
...,...,...,...,...,...,...,...,...,...,...,...
231,0974597002,Pocket lining: Cotton 100%,Slim fit,NaN,NaN,https://www2.hm.com/en_us/productpage.09745970...,0974597002,Denim blue,Slim Tapered Jeans,\r\n $29.99,2022-07-01 15:18:25
232,0974597003,"Shell: Cotton 98%, Spandex 2%",Slim fit,NaN,NaN,https://www2.hm.com/en_us/productpage.09745970...,0974597003,Black,Slim Tapered Jeans,\r\n $22.99,2022-07-01 15:18:25
233,0974597003,"Pocket lining: Polyester 65%, Cotton 35%",Slim fit,NaN,NaN,https://www2.hm.com/en_us/productpage.09745970...,0974597003,Black,Slim Tapered Jeans,\r\n $22.99,2022-07-01 15:18:25
234,0974597006,"Shell: Cotton 98%, Spandex 2%",Slim fit,NaN,NaN,https://www2.hm.com/en_us/productpage.09745970...,0974597006,Dark gray,Slim Tapered Jeans,\r\n $29.99,2022-07-01 15:18:25


## 2. DATA CLEANING 

In [ ]:
# delete $ from records
df['price'] = df['price'].apply(lambda x: x.replace('$', '')).str.strip()

# composition
df = df[~df['Composition'].str.contains('Pocket lining:', na = False)]
df = df[~df['Composition'].str.contains('Lining:', na = False)]
df = df[~df['Composition'].str.contains('Pocket:', na = False)]

# reset index
df = df.reset_index(drop = True)

# formats the values of the variable Composition
for i in df[df['Composition'].str.contains('Shell:', na = False)]['Composition']:
    # index
    line = df[df['Composition'] == i].index
    
    # extract only the compositon
    df.loc[line, 'Composition'] = i.split(': ')[1]

# change data type - id
df['Art. No.'] = df['Art. No.'].astype(int)

# change data type - price
df['price'] = df['price'].astype(float)

# change data type - date
df['web_scraping_date'] = pd.to_datetime(df['web_scraping_date'], format = '%Y-%m-%d %H:%M:%S')

# select features
df = df[['Art. No.', 'Composition', 'Fit', 'color_url', 'color_name', 'name', 'price', 'web_scraping_date']]

# rename 
df = df.rename(columns = {'Art. No.': 'id', 'Composition': 'composition', 'color_url': 'url', 'color_name': 'color',
                          'web_scraping_date': 'date', 'Fit': 'fit'})

## 3. DATABASE

### 3.1. Create table

In [ ]:
query_jeans_schema = """
                            CREATE TABLE IF NOT EXISTS jeans(
                                    id               INTEGER,
                                    composition      TEXT,
                                    fit              TEXT,              
                                    url              TEXT,
                                    color            TEXT,
                                    name             TEXT,
                                    price            REAL,
                                    date             TEXT
                            );
                    """

# create table
conn = sqlite3.connect('../data/database_jeans.sqlite')
cursor = conn.execute(query_jeans_schema)
conn.commit()

### 3.2. Insert data

In [ ]:
# organize the table
data_insert = df[['id', 'date', 'name', 'price', 'fit', 'composition', 'color', 'url']].copy()
    
# create database connection
conn = sqlite3.connect('../data/database_jeans.sqlite')

# insert data
data_insert.to_sql('jeans', con = conn, if_exists = 'append', index = False)